# Problema 3 

Considere que en una habitación con MxN espacios, hay P robots limpiadores reactivos. Cada uno de los robot limpiadores se comporta de la siguiente manera:

- Si la celda en la que se encuentra está sucia, entonces aspira durante 10 segundos.
- Si la celda está limpia, el robot elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas que esté sin otro robot) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda). Este movimiento dura 2 segundos.
- Si varios robots coinciden en alguna de las celdas, uno se queda en dicha celda y los otros se tiene que mover a alguna otra celda. 
- Quien se queda o quien se tiene que mover se elige aleatoriamente.
- Al inicio de la simulación, las posiciones de los robots son elegidas al azar, y de igual forma las posiciones están limpias o sucias aleatoriamente.

Realiza al menos 5 simulaciones en este entorno con diferente número de robots, y reporta lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias.
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.
- Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Qué comportamiento agregarías a los robots para que limpiaran con mayor velocidad?

In [26]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import IPython

# Random model
import random as rd

In [27]:
class RoomModel(ap.Model):

    def setup(self):

        # Create agents (floors)
        n_floors = int(self.p['Floors density'] * (self.p.size**2))
        floors = self.agents = ap.AgentList(self, n_floors)
        
        n_robots = int(self.p['P'] )
        robots = self.agents1 = ap.AgentList(self, n_robots)

        # Create grid (room)
        self.room = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.room.add_agents(floors, random=True, empty=True)
        self.room.add_agents(robots, random=True, empty=True)

        # Initiate a dynamic variable for all floors
        # Condition 0: Alive, 1: Burning, 2: Burned
        self.agents.condition = 0

        # Start a fire from the left side of the grid
        dirty_floors = self.room.agents[0:self.p.size, 0:2]
        dirty_floors.condition = 1

    def step(self):

        # Select burning floors
        clean_floors = self.agents1.select(self.agents.condition == 1)

        # Spread fire
        for floor in clean_floors:
            nex_floor = self.random.room(floor)
           # for nex_floor in self.room.nex_floors(floor):
            if nex_floor.condition == 0:
                nex_floor.condition = 1 # nex_floor starts burning
            while nex_floor.condition == 1 or nex_floor.time <= 10:
                nex_floor.time +=1
            if nex_floor.time >= 10:
                floor.condition = 2 # floor burns out

        # Stop simulation if no fire is left
        if len(clean_floors) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        clean_floors = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of cleaned floors',
                    clean_floors / len(self.agents))

In [28]:
parameters = {
    'Floors density': 0.4, # Percentage of grid covered by floors
    'P': 4, # Percentage of grid covered by floors
    'size': 50, # Height and length of the grid
    'steps': 60, # 1 minute
}

In [29]:
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a dirty room\n"
                 f"Time-sec: {model.t}, Floors left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = RoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

AttributeError: Agent (Obj 1001) has no attribute 'condition'.